In [1]:
from pymongo import MongoClient
import pymongo
import datetime
import numpy as np
import time
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm
from untitled import FirebaseMsg

In [2]:
import torch, gc # 쿠다 캐시 비우기
gc.collect()
torch.cuda.empty_cache()

In [3]:
client = MongoClient(host='114.70.235.37', port=27017)
print(client.list_database_names())
db = client['mydb'].xy_wj

['admin', 'config', 'local', 'mydb', 'test']


In [4]:
tmp_data= db.find().sort([("date", pymongo.DESCENDING)]).limit(2)
data = list(tmp_data)[0]
start_date = data["date"] - datetime.timedelta(seconds=5)

In [5]:
test = db.find({"date": {'$gte' : start_date, '$lt' : data["date"]}}).sort("human_index",pymongo.DESCENDING)
k = list(test)[0]
print(len(k))
print(k)
k = k["human_index"]
print(k)

23
{'_id': ObjectId('62d4d20609ce8b922931f257'), 'date': datetime.datetime(2022, 7, 18, 12, 22, 46, 826000), 'human_index': 0, 'v0x': 64.87140655517578, 'v0y': -44.68992614746094, 'v1x': 280.3585433959961, 'v1y': 306.9558868408203, 'v2x': 280.3585433959961, 'v2y': 306.9558868408203, 'v3x': 0.0, 'v3y': 0.0, 'v4x': 0.0, 'v4y': 0.0, 'v5x': 280.3585433959961, 'v5y': 306.9558868408203, 'v6x': 0.0, 'v6y': 0.0, 'v7x': 0.0, 'v7y': 0.0, 'v8x': 0.0, 'v8y': 0.0, 'v9x': 0.0, 'v9y': 0.0}
0


In [6]:
for i in range(k+1):
    result= db.find({"human_index":i}).sort([("date", pymongo.DESCENDING)]).limit(2)

In [7]:
print(list(result)[0])

{'_id': ObjectId('62d4d20c09ce8b922931f2a6'), 'date': datetime.datetime(2022, 7, 18, 12, 22, 52, 87000), 'human_index': 0, 'v0x': 67.8636474609375, 'v0y': -41.53831481933594, 'v1x': 289.7591552734375, 'v1y': 309.82508850097656, 'v2x': 289.7591552734375, 'v2y': 309.82508850097656, 'v3x': 0.0, 'v3y': 0.0, 'v4x': 0.0, 'v4y': 0.0, 'v5x': 289.7591552734375, 'v5y': 309.82508850097656, 'v6x': 0.0, 'v6y': 0.0, 'v7x': 0.0, 'v7y': 0.0, 'v8x': 0.0, 'v8y': 0.0, 'v9x': 0.0, 'v9y': 0.0}


In [8]:
def make_input():
    
    tmp_data= db.find().sort([("date", pymongo.DESCENDING)]).limit(2)
    data = list(tmp_data)[0]
    start_date = data["date"] - datetime.timedelta(seconds=5)
    
    max_people = db.find({"date": {'$gte' : start_date, '$lt' : data["date"]}}).sort("human_index",pymongo.DESCENDING).limit(2)
    k = list(max_people)[0]
    k = k["human_index"]
    tmp_loader = []
    for i in range(k+1):    
        result= db.find({"human_index":i}).sort([("date", pymongo.DESCENDING)]).limit(2)
    
        data = list(result)[0]
        start_date = data["date"] - datetime.timedelta(seconds=5)
        seq = list(db.find({"date": {'$gte' : start_date, '$lt' : data["date"]}}))
    
    
        list_data = list()
    
        for j in seq:
            del j['date']
            del j['_id']
            del j['human_index']
            
            list_data.append(list(j.values()))
        
        if(len(list_data) < 30):
            return
            
        index = list(range(5, len(list_data)-1, int(len(list_data)/30)))
        list_data = np.array(list_data, dtype=np.float32)
    
        y_false = np.array([0])
        list_data = list_data[index]
        list_data = np.array([list_data])
    
        tmp_data = TensorDataset(torch.from_numpy(list_data), torch.from_numpy(y_false))
        tmp_loader.append(DataLoader(tmp_data, shuffle=False, batch_size=batch_size))
    
    return tmp_loader

In [9]:
# dataloaders
batch_size = 32
device='cuda'

class SentimentLSTM(nn.Module):

    def __init__(self, seq_size, output_size, hidden_dim, n_layers):
        
        super(SentimentLSTM, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # embedding and LSTM layers
        self.lstm = nn.LSTM(seq_size, hidden_dim, n_layers, batch_first=True, dropout=0.2)
        
        # linear and sigmoid layer
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        batch_size = x.size(0)
        
        lstm_out, hidden = self.lstm(x)
        ## lstm_out.shape [batch, seq_len, hidden_dim]
        ## hidden [batch, hidden_dim]

        # fully connected layer        
        out = self.fc(lstm_out[:, -1, :])
        ## out.shape: [n_layer * n_direction, batch, output_size]

        # sigmoid function
        sig_out = self.sig(out)
        
        # return last sigmoid output and hidden state
        return sig_out, out

In [10]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        # nn.init.uniform_(param.data, -0.1, 0.1)

In [11]:
hidden_dim = 512
n_layers = 4
lr = 0.01
output_size = 1

model = SentimentLSTM(20, output_size, hidden_dim, n_layers)
model.to('cuda')

SentimentLSTM(
  (lstm): LSTM(20, 512, num_layers=4, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [12]:
model.load_state_dict(torch.load("model_weight.pth"))

<All keys matched successfully>

In [13]:
def predict(model, test_loader, sequence_length=30):
    
    model.eval()
    
    batch_size = 32
    acc_list = list()
    
    for inputs, labels in test_loader:
        inputs, labels = inputs.cuda(), labels.cuda()
        
        output, h = model(inputs)
        
        pred = torch.round(output.squeeze())
        
        accuracy = (pred==labels).sum().item() / 32
        acc_list.append(accuracy)

        #print('Prediction value, pre-rounding: ', output.item())

        # print custom response based on whether test_review is pos/neg
    print(acc_list)
    print(np.mean(acc_list))

In [14]:
def model_run(model, data_loader_list, sequence_length=30):
    
    model.eval()
    
    batch_size = 32
    acc_list = list()
    
    for data_loader in data_loader_list:
        for inputs, labels in data_loader:
            inputs, labels = inputs.cuda(), labels.cuda()

            output, h = model(inputs)

            pred = torch.round(output.squeeze())


            if int(pred) != 0:
                return True
            else:
                return False

In [15]:
fb_msg = FirebaseMsg()

In [18]:
def run(model, fb_msg, url):
    while True:
        data_loader_list = make_input()
        if data_loader_list is None:
            continue
        
        if model_run(model, data_loader_list):
            print("넘어짐 발생")
            fb_msg.falldown(url)
            time.sleep(3)

In [ ]:
run(model, fb_msg, "http://222.120.21.20:5000/stream?src=0")

넘어짐 발생
{'multicast_ids': [], 'success': 1, 'failure': 0, 'canonical_ids': 0, 'results': [], 'topic_message_id': 7876370032170001120}
넘어짐 발생
{'multicast_ids': [], 'success': 1, 'failure': 0, 'canonical_ids': 0, 'results': [], 'topic_message_id': 3221410308354969337}
